Берем отзывы за лето (из архива с материалами или предыдущего занятия)

Учим conv сеть для классификации\
Рассмотреть 2-а варианта сеточек:\
Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)\
Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/ \
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей\
сеточкой, что получилось лучше


In [ ]:
# pip install stop_words

In [ ]:
# pip install pymorphy2

In [238]:
import pandas as pd
import numpy as np

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

from gensim.models import Word2Vec
import gensim

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D 
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [ ]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [ ]:
df = pd.read_excel('/content/отзывы за лето.xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [ ]:
df.Rating.value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Заменим рейтинг на два класса

In [ ]:
df['Class'] = 0
df.loc[df.Rating > 2, 'Class'] = 1
df.loc[df.Rating <= 2, 'Class'] = 0
df.head(3)

,Rating,Content,Date,Class
0,5,It just works!,2017-08-14,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,1
2,5,Отлично все,2017-08-14,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('Class', axis=1), df.Class, test_size=.25, random_state=42, stratify=df.Class)

In [ ]:
X_train.size, X_test.size

(46482, 15495)

#### Предобработка текста

In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

X_train['Content_preproc'] = X_train['Content'].apply(preprocess_text)
X_test['Content_preproc'] = X_test['Content'].apply(preprocess_text)

In [ ]:
X_train.sample(5)

,Rating,Content,Date,Content_preproc
11880,5,Супер,2017-07-28,супер
10505,5,Мне всё нравится!!!,2017-07-30,нравиться
19136,5,Мейзу 5 нот приложение превратилось в тыкву!ис...,2017-06-28,мейза 5 нота приложение превратиться тыквуиспр...
1064,4,Удобно,2017-08-12,удобно
1621,4,В целом очень хорошо. Купил новый телефон и те...,2017-08-11,целое купить новый телефон теперьня мочь оплат...


#### Отфильтруем данные 
Создадим корпус частоиспользуемых токенов

In [ ]:
train_corpus = " ".join(X_train['Content_preproc'])

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train["Content_preproc"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test["Content_preproc"]], dtype=np.int32)

In [ ]:
x_train.shape

(15494, 40)

In [ ]:
x_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 3], dtype=int32)

### Keras model

In [ ]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 40, 128)           256000    
                                                                 
 conv1d_7 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 global_max_pooling1d_7 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_14 (Dense)            (None, 10)                1290      
                                                                 
 dense_15 (Dense)            (None, 2)                 22        
                                                                 
 activation_9 (Activation)   (None, 2)                 0         
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 7s 188ms/step - loss: 0.3593 - accuracy: 0.8318 - val_loss: 0.2537 - val_accuracy: 0.9019
Epoch 2/20
28/28 [==============================] - 4s 139ms/step - loss: 0.2252 - accuracy: 0.9039 - val_loss: 0.2141 - val_accuracy: 0.9045
Epoch 3/20
28/28 [==============================] - 3s 99ms/step - loss: 0.1772 - accuracy: 0.9248 - val_loss: 0.1974 - val_accuracy: 0.9123
Epoch 4/20
28/28 [==============================] - 3s 129ms/step - loss: 0.1483 - accuracy: 0.9394 - val_loss: 0.1980 - val_accuracy: 0.9110


In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

11/11 [==============================] - 0s 4ms/step - loss: 0.2191 - accuracy: 0.9045


Test score: 0.21912172436714172
Test accuracy: 0.9045498371124268


Загрузим данные для предобученной сетки

In [250]:
model_file = "model.txt"

embeddings_index = {}
with open(model_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word.split('_')[0]] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 195436 word vectors.


In [252]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [254]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len, weights=[embedding_matrix]))
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [255]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 40, 300)           600000    
                                                                 
 conv1d_11 (Conv1D)          (None, 38, 128)           115328    
                                                                 
 global_max_pooling1d_11 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_22 (Dense)            (None, 10)                1290      
                                                                 
 dense_23 (Dense)            (None, 2)                 22        
                                                                 
 activation_15 (Activation)  (None, 2)                 0         
                                                     

In [256]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [262]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 1s 44ms/step - loss: 0.0338 - accuracy: 0.9924 - val_loss: 0.2848 - val_accuracy: 0.9039
Epoch 2/20
28/28 [==============================] - 1s 53ms/step - loss: 0.0296 - accuracy: 0.9928 - val_loss: 0.2884 - val_accuracy: 0.9065


In [260]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

11/11 [==============================] - 0s 5ms/step - loss: 0.2327 - accuracy: 0.8978


Test score: 0.23272103071212769
Test accuracy: 0.8977735042572021


Модель с предобученными весами показала оценку не значительно ниже оценки модели без предобучения.